In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

pd.set_option('display.max_columns', 500)

In [2]:
train_df = pd.read_csv('kc_house_data_train.csv', index_col = 0)
display(train_df.shape)
train_df.head()



(17290, 21)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [3]:
train_df.drop(train_df.loc[train_df['bedrooms'].eq(33)].index, axis = 0, inplace=True)

In [4]:
train_df.shape

(17289, 21)

In [5]:
train_df['price_log'] = np.log(train_df['price'])

In [6]:
train_df['is_renovated'] = np.where(train_df['yr_renovated'].gt(0), 1, 0)
train_df['is_renovated'].value_counts()

0    16563
1      726
Name: is_renovated, dtype: int64

In [7]:
_grouped_qcut = pd.qcut(train_df.groupby(['zipcode'])['price'].mean(),
                        q=10, 
                        labels=['zip_lowest2','zip_lowest','zip_low2','zip_low','zip_medium2','zip_medium','zip_high2','zip_high','zip_highest2','zip_highest']).reset_index()
#                                'zip_lowest3','zip_lowest4','zip_low3','zip_low4','zip_medium3','zip_medium4','zip_high3','zip_hig4','zip_highest3','zip_highest4']).reset_index()
d = pd.Series(data= _grouped_qcut['price'].values,
              index = _grouped_qcut['zipcode']).to_dict()

# d

In [8]:
# pd.qcut(train_df.groupby(['zipcode'])['price'].mean(),
#                         q=3).value_counts()
#                         labels=['zip_lowest','zip_low','zip_medium','zip_high','zip_highest'])

In [9]:
train_df['zipcode_rating'] = train_df['zipcode'].map(d)
train_df['zipcode_rating'].value_counts()

zip_low2        2004
zip_low         1993
zip_highest2    1960
zip_high2       1960
zip_high        1908
zip_medium      1861
zip_lowest      1682
zip_medium2     1562
zip_lowest2     1321
zip_highest     1038
Name: zipcode_rating, dtype: int64

In [10]:
train_df['view'].value_counts()

0    15570
2      775
3      410
4      270
1      264
Name: view, dtype: int64

In [11]:
train_df['is_viewed'] = np.where(train_df['view'] > 0, 1,0)

In [12]:
train_df['sqft_living_log'] = np.log(train_df['sqft_living'])
train_df['sqft_lot_log'] = np.log(train_df['sqft_lot'])
train_df['sqft_above_log'] = np.log(train_df['sqft_above'])

In [13]:
train_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price_log,is_renovated,zipcode_rating,is_viewed,sqft_living_log,sqft_lot_log,sqft_above_log
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,12.807653,0,zip_low2,0,7.635304,9.093020,7.635304
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,13.670485,0,zip_high2,0,7.972466,8.814330,7.512071
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,13.852806,0,zip_highest2,1,8.234830,9.295876,8.234830
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,14.214287,0,zip_medium,1,8.425078,9.589325,8.425078
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,13.474428,0,zip_high,0,7.843849,8.589700,7.843849


In [14]:
train_df['is_basement'] = np.where(train_df['sqft_basement'].gt(0), 1, 0)

In [15]:
train_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price_log,is_renovated,zipcode_rating,is_viewed,sqft_living_log,sqft_lot_log,sqft_above_log,is_basement
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,12.807653,0,zip_low2,0,7.635304,9.093020,7.635304,0
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,13.670485,0,zip_high2,0,7.972466,8.814330,7.512071,1
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,13.852806,0,zip_highest2,1,8.234830,9.295876,8.234830,0
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,14.214287,0,zip_medium,1,8.425078,9.589325,8.425078,0
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,13.474428,0,zip_high,0,7.843849,8.589700,7.843849,0


In [16]:
train_df['sqft_living15_log'] = np.log(train_df['sqft_living15'])
train_df['sqft_lot15_log'] = np.log(train_df['sqft_lot15'])

In [17]:
train_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price_log,is_renovated,zipcode_rating,is_viewed,sqft_living_log,sqft_lot_log,sqft_above_log,is_basement,sqft_living15_log,sqft_lot15_log
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,12.807653,0,zip_low2,0,7.635304,9.093020,7.635304,0,7.779049,8.948976
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,13.670485,0,zip_high2,0,7.972466,8.814330,7.512071,1,7.770645,8.745603
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,13.852806,0,zip_highest2,1,8.234830,9.295876,8.234830,0,8.218787,9.178334
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,14.214287,0,zip_medium,1,8.425078,9.589325,8.425078,0,8.306472,9.562827
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,13.474428,0,zip_high,0,7.843849,8.589700,7.843849,0,7.718685,8.306472


In [18]:
train_df['lat_log'] = np.log(train_df['lat'])

In [19]:
train_df['long_log'] = np.log(train_df['long'] * -1)

In [20]:
train_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price_log,is_renovated,zipcode_rating,is_viewed,sqft_living_log,sqft_lot_log,sqft_above_log,is_basement,sqft_living15_log,sqft_lot15_log,lat_log,long_log
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,12.807653,0,zip_low2,0,7.635304,9.093020,7.635304,0,7.779049,8.948976,3.859440,4.805348
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,13.670485,0,zip_high2,0,7.972466,8.814330,7.512071,1,7.770645,8.745603,3.864478,4.806354
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,13.852806,0,zip_highest2,1,8.234830,9.295876,8.234830,0,8.218787,9.178334,3.862089,4.805078
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,14.214287,0,zip_medium,1,8.425078,9.589325,8.425078,0,8.306472,9.562827,3.864921,4.805888
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,13.474428,0,zip_high,0,7.843849,8.589700,7.843849,0,7.718685,8.306472,3.864191,4.804701


In [21]:
train_df['floors'].value_counts()

1.0    8605
2.0    6538
1.5    1524
3.0     475
2.5     140
3.5       7
Name: floors, dtype: int64

In [22]:
train_df['bath_cat'] = pd.cut(train_df['floors'], 
                              bins = [0, 1.51, 2.51, 3.76, 4.76,20], 
                              labels = ['0_1_bath','1.5_bath','2_2.5_bath','3_more_bath','more'])

In [23]:
train_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price_log,is_renovated,zipcode_rating,is_viewed,sqft_living_log,sqft_lot_log,sqft_above_log,is_basement,sqft_living15_log,sqft_lot15_log,lat_log,long_log,bath_cat
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,12.807653,0,zip_low2,0,7.635304,9.093020,7.635304,0,7.779049,8.948976,3.859440,4.805348,1.5_bath
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,13.670485,0,zip_high2,0,7.972466,8.814330,7.512071,1,7.770645,8.745603,3.864478,4.806354,0_1_bath
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,13.852806,0,zip_highest2,1,8.234830,9.295876,8.234830,0,8.218787,9.178334,3.862089,4.805078,1.5_bath
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,14.214287,0,zip_medium,1,8.425078,9.589325,8.425078,0,8.306472,9.562827,3.864921,4.805888,1.5_bath
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,13.474428,0,zip_high,0,7.843849,8.589700,7.843849,0,7.718685,8.306472,3.864191,4.804701,1.5_bath


In [24]:
train_df['yr_decade'] = pd.cut(x= train_df['yr_built'], 
       bins =[1899, 1909, 1919, 1929, 1939, 1949, 1959, 1969, 1979, 1989, 1999,2009,2019],
      labels= ['1900s', '1910s', '1920s', '1930s','1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s'])

In [25]:
train_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price_log,is_renovated,zipcode_rating,is_viewed,sqft_living_log,sqft_lot_log,sqft_above_log,is_basement,sqft_living15_log,sqft_lot15_log,lat_log,long_log,bath_cat,yr_decade
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,12.807653,0,zip_low2,0,7.635304,9.093020,7.635304,0,7.779049,8.948976,3.859440,4.805348,1.5_bath,1980s
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,13.670485,0,zip_high2,0,7.972466,8.814330,7.512071,1,7.770645,8.745603,3.864478,4.806354,0_1_bath,1970s
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,13.852806,0,zip_highest2,1,8.234830,9.295876,8.234830,0,8.218787,9.178334,3.862089,4.805078,1.5_bath,1990s
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,14.214287,0,zip_medium,1,8.425078,9.589325,8.425078,0,8.306472,9.562827,3.864921,4.805888,1.5_bath,1990s
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,13.474428,0,zip_high,0,7.843849,8.589700,7.843849,0,7.718685,8.306472,3.864191,4.804701,1.5_bath,2000s


In [26]:
train_df['grade'].value_counts()

7     7182
8     4848
9     2099
6     1648
10     893
11     319
5      192
12      72
4       22
13      11
3        2
1        1
Name: grade, dtype: int64

In [27]:
train_df['grade_cat'] = pd.cut(x= train_df['grade'],
      bins=[0,6,7,8,14],
      labels=['bad','bad-good','good','great'])

In [28]:
pd.qcut(train_df['bedrooms'], q=3)

0           (3.0, 4.0]
1          (4.0, 11.0]
2           (3.0, 4.0]
3        (-0.001, 3.0]
4        (-0.001, 3.0]
             ...      
17285      (4.0, 11.0]
17286    (-0.001, 3.0]
17287    (-0.001, 3.0]
17288    (-0.001, 3.0]
17289    (-0.001, 3.0]
Name: bedrooms, Length: 17289, dtype: category
Categories (3, interval[float64]): [(-0.001, 3.0] < (3.0, 4.0] < (4.0, 11.0]]

In [29]:
train_df['bedrooms'].value_counts()

3     7864
4     5488
2     2204
5     1283
6      229
1      160
7       30
0       12
8       10
9        5
10       3
11       1
Name: bedrooms, dtype: int64

In [30]:
train_df['bedrooms_cat'] = pd.cut(x=train_df['bedrooms'],
      bins=[-1,1.01,3.01,5.01,12],
      labels=['bed_0_1','bed_2_3','bed_4_5','bed_6_on'])

In [31]:
train_df['sqft_living_diff'] = train_df['sqft_living'] - train_df['sqft_living15']

In [32]:
train_df['sqft_lot_1/n_log'] = np.log(1/train_df['sqft_lot'])

In [33]:
train_df['sqft_lot15_1/n_log'] = np.log(1/train_df['sqft_lot15'])

In [34]:
train_df['building_age_log'] = np.log(2020 - train_df['yr_built'])

In [35]:
train_df['lat_cos'] = np.cos(train_df['lat'])
train_df['lon_cos'] = np.cos(train_df['long'])
train_df['lat_sin'] = np.sin(train_df['lat'])
train_df['lon_sin'] = np.sin(train_df['long'])

In [36]:
# train_df["rot45X"] = .707* train_df["long"] + .707* train_df["lat"] 
# train_df["rot45Y"] = .707* train_df["long"] - .707* train_df["lat"]

# train_df["rot30X"] = (1.732/2)* train_df["lat"] + (1./2)* train_df["long"] 
# train_df["rot30Y"] = (1.732/2)* train_df["long"] - (1./2)* train_df["lat"]

# train_df["rot60X"] = (1./2)* train_df["lat"] + (1.732/2)* train_df["long"] 
# train_df["rot60Y"] = (1./2)* train_df["long"] - (1.732/2)* train_df["lat"]

# train_df["radial_r"] = np.sqrt( np.power(train_df["long"],2) + np.power(train_df["lat"],2) )

In [37]:
pd.concat([train_df, pd.get_dummies(train_df['bedrooms'])], axis=1)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price_log,is_renovated,zipcode_rating,is_viewed,sqft_living_log,sqft_lot_log,sqft_above_log,is_basement,sqft_living15_log,sqft_lot15_log,lat_log,long_log,bath_cat,yr_decade,grade_cat,bedrooms_cat,sqft_living_diff,sqft_lot_1/n_log,sqft_lot15_1/n_log,building_age_log,lat_cos,lon_cos,lat_sin,lon_sin,0,1,2,3,4,5,6,7,8,9,10,11
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,12.807653,0,zip_low2,0,7.635304,9.093020,7.635304,0,7.779049,8.948976,3.859440,4.805348,1.5_bath,1980s,good,bed_4_5,-320,-9.093020,-8.948976,3.526361,-0.950824,-0.935857,-0.309731,-0.352380,0,0,0,0,1,0,0,0,0,0,0,0
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,13.670485,0,zip_high2,0,7.972466,8.814330,7.512071,1,7.770645,8.745603,3.864478,4.806354,0_1_bath,1970s,good,bed_4_5,530,-8.814330,-8.745603,3.761200,-0.850158,-0.972020,-0.526527,-0.234898,0,0,0,0,0,1,0,0,0,0,0,0
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,13.852806,0,zip_highest2,1,8.234830,9.295876,8.234830,0,8.218787,9.178334,3.862089,4.805078,1.5_bath,1990s,great,bed_4_5,60,-9.295876,-9.178334,3.135494,-0.904449,-0.923721,-0.426582,-0.383066,0,0,0,0,1,0,0,0,0,0,0,0
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,14.214287,0,zip_medium,1,8.425078,9.589325,8.425078,0,8.306472,9.562827,3.864921,4.805888,1.5_bath,1990s,great,bed_2_3,510,-9.589325,-9.562827,3.401197,-0.838860,-0.957060,-0.544347,-0.289892,0,0,0,1,0,0,0,0,0,0,0,0
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,13.474428,0,zip_high,0,7.843849,8.589700,7.843849,0,7.718685,8.306472,3.864191,4.804701,1.5_bath,2000s,great,bed_2_3,300,-8.589700,-8.306472,2.772589,-0.857292,-0.905129,-0.514831,-0.425137,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,627300195,20150303T000000,750000.0,5,2.50,3240,9960,1.0,0,1,3,8,2020,1220,1958,0,98008,47.5858,-122.112,2730,10400,13.527828,0,zip_high,1,8.083329,9.206332,7.610853,1,7.912057,9.249561,3.862534,4.804939,0_1_bath,1950s,good,bed_4_5,510,-9.206332,-9.249561,4.127134,-0.895203,-0.917076,-0.445659,-0.398713,0,0,0,0,0,1,0,0,0,0,0,0
17286,8819900270,20140520T000000,440000.0,2,1.75,1300,4000,2.0,0,0,3,7,1300,0,1948,0,98105,47.6687,-122.288,1350,4013,12.994530,0,zip_highest2,0,7.170120,8.294050,7.170120,0,7.207860,8.297294,3.864275,4.806379,1.5_bath,1940s,bad-good,bed_2_3,-50,-8.294050,-8.297294,4.276666,-0.855226,-0.972720,-0.518256,-0.231981,0,0,1,0,0,0,0,0,0,0,0,0
17287,3816300095,20140514T000000,310000.0,3,1.00,1050,9876,1.0,0,0,3,7,1050,0,1953,0,98028,47.7635,-122.262,1760,9403,12.644328,0,zip_medium2,0,6.956545,9.197863,6.956545,0,7.473069,9.148784,3.866262,4.806166,0_1_bath,1950s,bad-good,bed_2_3,-710,-9.197863,-9.148784,4.204693,-0.802328,-0.966361,-0.596883,-0.257190,0,0,0,1,0,0,0,0,0,0,0,0
17288,122069107,20141204T000000,427500.0,3,1.50,1900,43186,1.5,0,0,4,7,1300,600,1971,0,98038,47.4199,-121.990,2080,108028,12.965710,0,zip_low2,0,7.549609,10.673272,7.170120,1,7.640123,11.590146,3.859042,4.803939,0_1_bath,1970s,bad-good,bed_2_3,-180,-10.673272,-11.590146,3.891820,-0.956508,-0.861737,-0.291706,-0.507356,0,0,0,1,0,0,0,0,0,0,0,0


In [38]:
# pd.get_dummies(train_df['bedrooms'], prefix_sep= '_')

In [39]:
_train_df = pd.get_dummies(train_df, 
#                prefix=['bedrooms', 'floors','condition', 'grade', 'zipcode_rating','yr_decade','bath_cat'], 
               columns =['bedrooms_cat','floors','condition','zipcode_rating','yr_decade','bath_cat'])

In [40]:
_train_df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,waterfront,view,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price_log,is_renovated,is_viewed,sqft_living_log,sqft_lot_log,sqft_above_log,is_basement,sqft_living15_log,sqft_lot15_log,lat_log,long_log,grade_cat,sqft_living_diff,sqft_lot_1/n_log,sqft_lot15_1/n_log,building_age_log,lat_cos,lon_cos,lat_sin,lon_sin,bedrooms_cat_bed_0_1,bedrooms_cat_bed_2_3,bedrooms_cat_bed_4_5,bedrooms_cat_bed_6_on,floors_1.0,floors_1.5,floors_2.0,floors_2.5,floors_3.0,floors_3.5,condition_1,condition_2,condition_3,condition_4,condition_5,zipcode_rating_zip_high,zipcode_rating_zip_high2,zipcode_rating_zip_highest,zipcode_rating_zip_highest2,zipcode_rating_zip_low,zipcode_rating_zip_low2,zipcode_rating_zip_lowest,zipcode_rating_zip_lowest2,zipcode_rating_zip_medium,zipcode_rating_zip_medium2,yr_decade_1900s,yr_decade_1910s,yr_decade_1920s,yr_decade_1930s,yr_decade_1940s,yr_decade_1950s,yr_decade_1960s,yr_decade_1970s,yr_decade_1980s,yr_decade_1990s,yr_decade_2000s,yr_decade_2010s,bath_cat_0_1_bath,bath_cat_1.5_bath,bath_cat_2_2.5_bath,bath_cat_3_more_bath,bath_cat_more
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,0,0,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,12.807653,0,0,7.635304,9.093020,7.635304,0,7.779049,8.948976,3.859440,4.805348,good,-320,-9.093020,-8.948976,3.526361,-0.950824,-0.935857,-0.309731,-0.352380,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,0,0,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,13.670485,0,0,7.972466,8.814330,7.512071,1,7.770645,8.745603,3.864478,4.806354,good,530,-8.814330,-8.745603,3.761200,-0.850158,-0.972020,-0.526527,-0.234898,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,0,2,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,13.852806,0,1,8.234830,9.295876,8.234830,0,8.218787,9.178334,3.862089,4.805078,great,60,-9.295876,-9.178334,3.135494,-0.904449,-0.923721,-0.426582,-0.383066,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,0,2,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,14.214287,0,1,8.425078,9.589325,8.425078,0,8.306472,9.562827,3.864921,4.805888,great,510,-9.589325,-9.562827,3.401197,-0.838860,-0.957060,-0.544347,-0.289892,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,0,0,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,13.474428,0,0,7.843849,8.589700,7.843849,0,7.718685,8.306472,3.864191,4.804701,great,300,-8.589700,-8.306472,2.772589,-0.857292,-0.905129,-0.514831,-0.425137,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,627300195,20150303T000000,750000.0,5,2.50,3240,9960,0,1,8,2020,1220,1958,0,98008,47.5858,-122.112,2730,10400,13.527828,0,1,8.083329,9.206332,7.610853,1,7.912057,9.249561,3.862534,4.804939,good,510,-9.206332,-9.249561,4.127134,-0.895203,-0.917076,-0.445659,-0.398713,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
17286,8819900270,20140520T000000,440000.0,2,1.75,1300,4000,0,0,7,1300,0,1948,0,98105,47.6687,-122.288,1350,4013,12.994530,0,0,7.170120,8.294050,7.170120,0,7.207860,8.297294,3.864275,4.806379,bad-good,-50,-8.294050,-8.297294,4.276666,-0.855226,-0.972720,-0.518256,-0.231981,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
17287,3816300095,20140514T000000,

In [44]:
X = _train_df.drop(columns=['id','date','price','bathrooms','view','sqft_basement','yr_built','yr_renovated','zipcode','price_log','sqft_living','sqft_lot', 'sqft_lot_log',
                           'sqft_above','sqft_above_log','sqft_living15','sqft_lot15','sqft_living15_log', 'sqft_lot15_log', 'lat','long','bedrooms','grade_cat']).copy()
                           #'lat_log','long_log']) #'sqft_living15_log', 'sqft_lot15_log'
y = _train_df['price_log']

In [45]:
# _X = X.drop(columns = ['bedrooms_0','floors_1.0','condition_1','grade_1','zipcode_rating_zip_high','yr_decade_1900s','bath_cat_0_1_bath'])

In [46]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(X)
poly2_columns = poly_2.get_feature_names(X.columns)
df_poly2 = pd.DataFrame(poly2_data, columns = poly2_columns)
df_poly2

,waterfront,grade,is_renovated,is_viewed,sqft_living_log,is_basement,lat_log,long_log,sqft_living_diff,sqft_lot_1/n_log,sqft_lot15_1/n_log,building_age_log,lat_cos,lon_cos,lat_sin,lon_sin,bedrooms_cat_bed_0_1,bedrooms_cat_bed_2_3,bedrooms_cat_bed_4_5,bedrooms_cat_bed_6_on,floors_1.0,floors_1.5,floors_2.0,floors_2.5,floors_3.0,floors_3.5,condition_1,condition_2,condition_3,condition_4,condition_5,zipcode_rating_zip_high,zipcode_rating_zip_high2,zipcode_rating_zip_highest,zipcode_rating_zip_highest2,zipcode_rating_zip_low,zipcode_rating_zip_low2,zipcode_rating_zip_lowest,zipcode_rating_zip_lowest2,zipcode_rating_zip_medium,zipcode_rating_zip_medium2,yr_decade_1900s,yr_decade_1910s,yr_decade_1920s,yr_decade_1930s,yr_decade_1940s,yr_decade_1950s,yr_decade_1960s,yr_decade_1970s,yr_decade_1980s,yr_decade_1990s,yr_decade_2000s,yr_decade_2010s,bath_cat_0_1_bath,bath_cat_1.5_bath,bath_cat_2_2.5_bath,bath_cat_3_more_bath,bath_cat_more,waterfront^2,waterfront grade,waterfront is_renovated,waterfront is_viewed,waterfront sqft_living_log,waterfront is_basement,waterfront lat_log,waterfront long_log,waterfront sqft_living_diff,waterfront sqft_lot_1/n_log,waterfront sqft_lot15_1/n_log,waterfront building_age_log,waterfront lat_cos,waterfront lon_cos,waterfront lat_sin,waterfront lon_sin,waterfront bedrooms_cat_bed_0_1,waterfront bedrooms_cat_bed_2_3,waterfront bedrooms_cat_bed_4_5,waterfront bedrooms_cat_bed_6_on,waterfront floors_1.0,waterfront floors_1.5,waterfront floors_2.0,waterfront floors_2.5,waterfront floors_3.0,waterfront floors_3.5,waterfront condition_1,waterfront condition_2,waterfront condition_3,waterfront condition_4,waterfront condition_5,waterfront zipcode_rating_zip_high,waterfront zipcode_rating_zip_high2,waterfront zipcode_rating_zip_highest,waterfront zipcode_rating_zip_highest2,waterfront zipcode_rating_zip_low,waterfront zipcode_rating_zip_low2,waterfront zipcode_rating_zip_lowest,waterfront zipcode_rating_zip_lowest2,waterfront zipcode_rating_zip_medium,waterfront zipcode_rating_zip_medium2,waterfront yr_decade_1900s,waterfront yr_decade_1910s,waterfront yr_decade_1920s,waterfront yr_decade_1930s,waterfront yr_decade_1940s,waterfront yr_decade_1950s,waterfront yr_decade_1960s,waterfront yr_decade_1970s,waterfront yr_decade_1980s,waterfront yr_decade_1990s,waterfront yr_decade_2000s,waterfront yr_decade_2010s,waterfront bath_cat_0_1_bath,waterfront bath_cat_1.5_bath,waterfront bath_cat_2_2.5_bath,waterfront bath_cat_3_more_bath,waterfront bath_cat_more,grade^2,grade is_renovated,grade is_viewed,grade sqft_living_log,grade is_basement,grade lat_log,grade long_log,grade sqft_living_diff,grade sqft_lot_1/n_log,grade sqft_lot15_1/n_log,grade building_age_log,grade lat_cos,grade lon_cos,grade lat_sin,grade lon_sin,grade bedrooms_cat_bed_0_1,grade bedrooms_cat_bed_2_3,grade bedrooms_cat_bed_4_5,grade bedrooms_cat_bed_6_on,grade floors_1.0,grade floors_1.5,grade floors_2.0,grade floors_2.5,grade floors_3.0,grade floors_3.5,grade condition_1,grade condition_2,grade condition_3,grade condition_4,grade condition_5,grade zipcode_rating_zip_high,grade zipcode_rating_zip_high2,grade zipcode_rating_zip_highest,grade zipcode_rating_zip_highest2,grade zipcode_rating_zip_low,grade zipcode_rating_zip_low2,grade zipcode_rating_zip_lowest,grade zipcode_rating_zip_lowest2,grade zipcode_rating_zip_medium,grade zipcode_rating_zip_medium2,grade yr_decade_1900s,grade yr_decade_1910s,grade yr_decade_1920s,grade yr_decade_1930s,grade yr_decade_1940s,grade yr_decade_1950s,grade yr_decade_1960s,grade yr_decade_1970s,grade yr_decade_1980s,grade yr_decade_1990s,grade yr_decade_2000s,grade yr_decade_2010s,grade bath_cat_0_1_bath,grade bath_cat_1.5_bath,grade bath_cat_2_2.5_bath,grade bath_cat_3_more_bath,grade bath_cat_more,is_renovated^2,is_renovated is_viewed,is_renovated sqft_living_log,is_renovated is_basement,is_renovated lat_log,is_renovated long_log,is_renovated sqft_living_diff,is_renovated sqft_lot_1/n_log,is_renovated sqft_lot15_1/n_log,is

In [47]:
# df_poly2

In [48]:
# list(df_poly2.columns)

In [49]:
_X = df_poly2.drop(columns=['floors_1.0','condition_1','zipcode_rating_zip_high','yr_decade_1900s','bath_cat_more','bedrooms_cat_bed_6_on'], axis=1)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(df_poly2, y, random_state=42, test_size=0.2)

In [51]:
selector = SelectKBest(f_regression, k=300)
selector.fit(X_train, y_train)

C:\Users\albert\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
C:\Users\albert\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\albert\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\albert\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


SelectKBest(k=300, score_func=<function f_regression at 0x00000000100224C8>)

In [52]:
selected_columns = X_train.columns[selector.get_support()]

In [53]:
list(selected_columns)

['waterfront',
 'grade',
 'is_viewed',
 'sqft_living_log',
 'is_basement',
 'lat_log',
 'sqft_living_diff',
 'lat_cos',
 'lat_sin',
 'bedrooms_cat_bed_2_3',
 'bedrooms_cat_bed_4_5',
 'floors_1.0',
 'floors_2.0',
 'zipcode_rating_zip_highest',
 'zipcode_rating_zip_highest2',
 'zipcode_rating_zip_low2',
 'zipcode_rating_zip_lowest',
 'zipcode_rating_zip_lowest2',
 'bath_cat_0_1_bath',
 'bath_cat_1.5_bath',
 'waterfront^2',
 'waterfront grade',
 'waterfront is_viewed',
 'waterfront sqft_living_log',
 'waterfront lat_log',
 'waterfront long_log',
 'waterfront sqft_lot_1/n_log',
 'waterfront sqft_lot15_1/n_log',
 'waterfront building_age_log',
 'waterfront lat_cos',
 'waterfront lon_cos',
 'waterfront lat_sin',
 'waterfront lon_sin',
 'grade^2',
 'grade is_viewed',
 'grade sqft_living_log',
 'grade is_basement',
 'grade lat_log',
 'grade long_log',
 'grade sqft_living_diff',
 'grade sqft_lot_1/n_log',
 'grade sqft_lot15_1/n_log',
 'grade building_age_log',
 'grade lat_cos',
 'grade lon_cos'

In [54]:
lm_kbest = LinearRegression()
lm_kbest = lm_kbest.fit(X_train[selected_columns], y_train)
y_train_kbest = lm_kbest.predict(X_train[selected_columns])
np.sqrt(metrics.mean_squared_error(np.exp(y_train), np.exp(y_train_kbest)))

118995.92908158831

In [55]:
y_kbest = lm_kbest.predict(X_test[selected_columns])
np.sqrt(metrics.mean_squared_error(np.exp(y_test), np.exp(y_kbest)))

129138.83038549889

In [56]:
selected_columns

Index(['waterfront', 'grade', 'is_viewed', 'sqft_living_log', 'is_basement',
       'lat_log', 'sqft_living_diff', 'lat_cos', 'lat_sin',
       'bedrooms_cat_bed_2_3',
       ...
       'zipcode_rating_zip_highest2 bath_cat_1.5_bath',
       'zipcode_rating_zip_low2^2',
       'zipcode_rating_zip_low2 bath_cat_0_1_bath',
       'zipcode_rating_zip_lowest^2',
       'zipcode_rating_zip_lowest bath_cat_0_1_bath',
       'zipcode_rating_zip_lowest2^2',
       'zipcode_rating_zip_lowest2 yr_decade_1960s',
       'zipcode_rating_zip_lowest2 bath_cat_0_1_bath', 'bath_cat_0_1_bath^2',
       'bath_cat_1.5_bath^2'],
      dtype='object', length=300)

In [ ]:
from sklearn.linear_model import Lasso

X_train, X_test, y_train, y_test = train_test_split(df_poly2[selected_columns], y, random_state=42, test_size=0.2)


lasso = Lasso(alpha=.01, normalize=True)
lasso.fit(X_train, y_train)

y_train_pred_lasso = lasso.predict(X_train)
y_test_pred_lasso = lasso.predict(X_test)

In [ ]:
np.sqrt(metrics.mean_squared_error(np.exp(y_test), np.exp(y_test_pred_lasso)))

In [ ]:
np.sqrt(metrics.mean_squared_error(np.exp(y_train), np.exp(y_train_pred_lasso)))

In [ ]:
df_poly2[selected_columns]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split()